In [1]:
import os
import sys
import open3d as o3d
import numpy as np
import cv2
import copy
import matplotlib.pyplot as plt
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/milestone_202110'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.project_config import *
from utils.streaming import *
from utils.detection_util import *

CONNECT_CAM = False # True
ENABLE_DETECT = True
CONNECT_INDY = False # True
CONNECT_MOBILE = False # True 
SHOW_MOTION_RVIZ = False

ip_cur =  get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"
CAM_HOST = '192.168.0.10'

print("Current PC IP: {}".format(ip_cur))
print("Mobile ROB IP: {}".format(MOBILE_IP))
print("CAM SERVER IP: {}".format(CAM_HOST))

Current PC IP: 192.168.0.40
Mobile ROB IP: 192.168.0.102
CAM SERVER IP: 192.168.0.10


## Attach to detection server

In [2]:
if ENABLE_DETECT:
    attacth_to_server()

## 침대 인식

In [3]:
# Coordinate of stl model importing

# bed_model = o3d.io.read_triangle_mesh(MODEL_DIR + '/bed/bed.STL')
# # bed_model = o3d.io.read_triangle_mesh(MODEL_DIR + '/bed/bed_floor_centered_m_scale.stl')
# bed_model.vertices = o3d.utility.Vector3dVector(
#     np.asarray(bed_model.vertices) * np.array([1 / 1000.0, 1 / 1000.0, 1 / 1000.0]))
# FOR_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0])
# o3d.visualization.draw_geometries([bed_model, FOR_origin])

## Get image

In [4]:
CAM_HOST = '192.168.0.40'

In [5]:
if CONNECT_CAM:
    # rdict = send_recv_demo_cam({1:1}, host=CAM_HOST)
    rdict = stream_capture_image(ImageType.FirstView, 0, host=CAM_HOST)
    cam_intrins, d_scale = [rdict[key] for key in ["intrins", "depth_scale"]]
    set_cam_params(cam_intrins, d_scale)
else:
    cam_intrins = [1280, 720, 899.05322265625,  899.21044921875, 654.8836669921875, 352.9295654296875]
    d_scale = 0.0002500000118743628
    set_cam_params(cam_intrins, d_scale)

In [33]:
if CONNECT_CAM:
    bed_color_path = SAVE_DIR + '/bed.jpg'
    bed_depth_path = SAVE_DIR + '/bed.png'
else:
    bed_color_path = EXP_IMG_DIR + '/top_table_check.jpg'
    bed_depth_path = EXP_IMG_DIR + '/top_table_check.png'

# bed_color_path = "scripts/milestone_202110/save_img/526.jpg"
# bed_depth_path = "scripts/milestone_202110/save_img/526.png"

# Read color, depth image file, keep 16bit information
color_img_read = cv2.imread(bed_color_path, flags=cv2.IMREAD_UNCHANGED)
depth_img_read = cv2.imread(bed_depth_path, flags=cv2.IMREAD_UNCHANGED)


# Output of inference(mask for detected table)
mask_out = detect_from_server(color_img_read)

# If bed is not detected, then pass below detection part
test = np.empty((720,1280), dtype=bool)
test[:,:] = False
if not np.array_equal(mask_out, test):
    plt.imshow(mask_out)
    
    # Crop masking part
    vis_mask = (mask_out * 255).astype('uint8')
    color_instance = cv2.bitwise_and(color_img_read, color_img_read, mask=vis_mask).astype(np.uint16)
    depth_instance = cv2.bitwise_and(depth_img_read, depth_img_read, mask=vis_mask).astype(np.uint16)
    cv2.imwrite(CROP_DIR + '/bed_crop.jpg', color_instance)
    cv2.imwrite(CROP_DIR + '/bed_crop.png', depth_instance)
    
#     set_cam_params(cam_intrins, d_scale)
    ICP_result_bed = process_bed_detection()

In [31]:
test = np.empty((720,1280), dtype=bool)
test[:,:] = False
if not np.array_equal(mask_out, test):
    plt.imshow(mask_out)
    
    # Crop masking part
    vis_mask = (mask_out * 255).astype('uint8')
    color_instance = cv2.bitwise_and(color_img_read, color_img_read, mask=vis_mask).astype(np.uint16)
    depth_instance = cv2.bitwise_and(depth_img_read, depth_img_read, mask=vis_mask).astype(np.uint16)
    cv2.imwrite(CROP_DIR + '/bed_crop.jpg', color_instance)
    cv2.imwrite(CROP_DIR + '/bed_crop.png', depth_instance)
    
#     set_cam_params(cam_intrins, d_scale)
    ICP_result_bed = process_bed_detection()

In [6]:
# 인식한 결과를 바탕으로 geometry 추가
T_co = ICP_result_bed
T_bc = viewpoint.get_tf(list2dict(VIEW_POSE, gscene.joint_names))
T_bo = np.matmul(T_bc, T_co)

# geometry 추가
# how to import gemoetry from stl file?

NameError: name 'viewpoint' is not defined

## 상두대 인식

In [15]:
# Coordinate of stl model importing

# top_table_model = o3d.io.read_triangle_mesh(MODEL_DIR + '/top_table/top_table.STL')
# top_table_model.vertices = o3d.utility.Vector3dVector(
#     np.asarray(top_table_model.vertices) * np.array([1 / 1000.0, 1 / 1000.0, 1 / 1000.0]))
# FOR_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0])
# o3d.visualization.draw_geometries([top_table_model, FOR_origin])

## Move and Scan

In [10]:
if CONNECT_CAM:
    color_path = SAVE_DIR + '/bed.jpg'
    depth_path = SAVE_DIR + '/bed.png'
else:
    color_path = EXP_IMG_DIR + '/513.jpg'
    depth_path = EXP_IMG_DIR + '/513.jpg'

# Robot이 어느쪽에서 찍었는 지 "LEFT" or "RIGHT"만 넣어주면 됨
TOP_TABLE_LOCATION = check_location_top_table(color_path, depth_path, ROBOT_LOCATION)

In [ ]:
# if CONNECT_CAM:
#     rdict = stream_capture_image(ImageType.FirstView, 1, host=CAM_HOST)
#     check_color_path = SAVE_DIR + '/top_table_check.jpg'
#     check_depth_path = SAVE_DIR + '/top_table_check.png'
# else:
#     check_color_path = EXP_IMG_DIR + '/top_table_check.jpg'
#     check_depth_path = EXP_IMG_DIR + '/top_table_check.png'

# # Check whether top table exist or not at scanning location
# check_top_table = check_top_table_exist(check_color_path, check_depth_path)

In [5]:
# 인식한 침대 결과를 기반으로 대략적인 위치로 모바일 로봇 이동하거나,적당한 위치로 이동한 후 
# 스캔 결과를 treat 해서 여기에 상두대가 있는 지 없는지 판단
# 인식한 침대 결과 기반으로 이동한 후, 위아래로 이미지 스캔하는 동작 + 스캔하면서 이미지 저장
# 이미지 저장 시작 - 모션 시작 - 모션 끝 - 이미지 저장 끝 (위아래 왕복 모션이 필요 loop closure detection 때문)
# for문 / while문으로 이미지 연속으로 저장 가능
for i in range(50):
    time.sleep(0.01)
    rdict = stream_capture_image(ImageType.CloseView, i, host=CAM_HOST)

In [34]:
if not CONNECT_CAM:
    COLOR_PATH = EXP_IMG_DIR + "/top_table/color"
    DEPTH_PATH = EXP_IMG_DIR + "/top_table/depth"
    INTRINSIC_PATH = EXP_IMG_DIR + "/top_table"

save_intrinsic_as_json(INTRINSIC_PATH + "/intrinsic.json")
INTRINSIC_PATH = INTRINSIC_PATH + "/intrinsic.json"

In [12]:
from boost_reconstruction.reconstruction_interface import reconstruction_interface_py as rci
rci.getReconstruction(COLOR_PATH, DEPTH_PATH, INTRINSIC_PATH, MILESTONE_DIR)

'========= Complete 3D Reconstruction ========='

In [15]:
pcd_top_table = o3d.io.read_point_cloud(MILESTONE_DIR + "/pcd_no.ply")
FOR_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0])
o3d.visualization.draw_geometries([pcd_top_table, FOR_origin])

In [ ]:
ICP_result_top_table = process_top_table_detection()

In [ ]:
# 인식한 결과를 바탕으로 geometry 추가
# 이때 pcd의 coordinate, 즉 카메라 시점은 첫번째 이미지를 찍었을 때의 시점임. 유의 필요
T_co = ICP_result_top_table
T_bc = viewpoint.get_tf(list2dict(VIEW_POSE, gscene.joint_names))
T_bo = np.matmul(T_bc, T_co)

# geometry 추가
# how to import gemoetry from stl file?

### 배게 인식

In [6]:
import pickle

with open(SAVE_DIR + '/bed_close_shots/bed_close_4.pkl', 'rb') as f:
    data = pickle.load(f)

Q = data["Q"]
T_rb = data["T_robot_bed"]

color_instance = data["image_dict"]["color"]
depth_instance = data["image_dict"]["depth"]

pcd = make_pcd(color_instance, depth_instance)

o3d.visualization.draw_geometries([pcd])